## 01 compile relevant EVs data from EUAFO

## Attempt at webscraping data

In [1]:
import pandas as pd

### 1. Import EV Fleet Data
* Data is freely available on webpages on EUAFO, but not released as readily downloadable csv files.

### importing vehicle charge price data for each country
- after inspecting underlying code here: https://alternative-fuels-observatory.ec.europa.eu/consumer-portal/electric-vehicle-recharging-prices
which lists ad-hoc charge prices for various common electric vehicles (price for averge customer walking up to charge a vehicle), the following code blocks take the underlying data used for this calculation, and apply the same calculation to get the prices

In [2]:
# URLs (you can update these with latest tokens if needed)
country_url = "https://alternative-fuels-observatory.ec.europa.eu/sites/default/files/csv/custom-graphs/country_data.csv"
model_url = "https://alternative-fuels-observatory.ec.europa.eu/sites/default/files/csv/custom-graphs/model_data.csv"

In [ ]:
import requests
import os

# URL of the CSV
url = "https://alternative-fuels-observatory.ec.europa.eu/sites/default/files/csv/custom-graphs/model_data.csv"

# Path on your device where you want to save it
save_dir = "/Users/katehodges/Desktop/Applications/Portfolio/EVs Proj/data/01 raw" 
filename = "model_data.csv"
save_path = os.path.join(save_dir, filename)

# Add headers to avoid being blocked
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Download and save
response = requests.get(url, headers=headers)
response.raise_for_status()  # raises error if download failed

with open(save_path, "wb") as f:
    f.write(response.content)

print(f"Success: CSV saved to {save_path}")


✅ CSV saved to /Users/katehodges/Desktop/Applications/Portfolio/EVs Proj/data/01 raw/model_data.csv


##### calculation
To estimate EV recharging prices by country, I replicated the methodology used by the Alternative Fuels Observatory (AFO). Based on their web interface and underlying dataset, I calculated the ad hoc cost of charging a vehicle using public DC fast chargers. The cost was computed as:

Cost (€) = Energy charged (kWh) × ad hoc fee (€/kWh)
Cost (€)=Energy charged (kWh)×ad hoc fee (€/kWh)
This simplification aligns with the AFO’s displayed estimates and enables scalable, automated comparison across countries and models using publicly available data.

In [4]:
country_data = pd.read_csv('/Users/katehodges/Desktop/Applications/Portfolio/EVs Proj/data/01 raw/country_data.csv', sep = ';')

model_data = pd.read_csv('/Users/katehodges/Desktop/Applications/Portfolio/EVs Proj/data/01 raw/model_data.csv', sep = ';')

In [53]:

# clean names
country_data.columns = (
    country_data.columns
    .str.replace(r'\s+', '_', regex=True)    # replace all whitespace with underscore
    .str.replace(r'_+$', '', regex=True)     # remove trailing underscores
    .str.replace(r'^_+', '', regex=True)     # remove leading underscores
    .str.lower()                             # convert to lowercase
    .str.strip()                            # just in case, strip spaces (optional)
)

country_data

,country_code,country_name,vat,min_energy_fee_dc,min_time_fee_dc,max_energy_fee_dc,max_time_fee_dc,adhoc_energy_fee_dc,adhoc_time_fee_dc
0,AT,Austria,20%,0.24,0.008,0.82,1.117,0.516,0.683000
1,BE,Belgium,21%,0.23,0.008,0.88,0.370,0.553,0.126000
2,BG,Bulgaria,20%,0.39,0.050,0.46,0.336,0.485,0.170000
3,CH,Switzerland,8%,0.32,0.083,0.70,0.167,0.525,0.122167
4,CY,Cyprus,19%,0.24,0.050,1.79,0.370,0.650,0.170000
5,CZ,Czech Republic,21%,0.24,0.050,0.66,0.370,0.361,0.078670
6,DE,Germany,19%,0.16,0.008,5.15,0.792,0.590,0.086000
7,DK,Denmark,25%,0.24,0.083,0.66,0.167,0.582,0.226167
8,EE,Estonia,20%,0.28,0.250,0.79,0.480,0.660,0.170000
9,ES,Spain,21%,0.21,0.041,1.83,0.413,0.470,0.017000


In [54]:
# clean names
model_data.columns = (
    model_data.columns
    .str.replace(r'\s+', '_', regex=True)    # replace all whitespace with underscore
    .str.replace(r'_+$', '', regex=True)     # remove trailing underscores
    .str.replace(r'^_+', '', regex=True)     # remove leading underscores
    .str.lower()                             # convert to lowercase
    .str.strip()                            # just in case, strip spaces (optional)
)

model_data

,model,"fastcharge_time_(min,_dc)",charged_km,power_charged_(kw),segment,model_clean
0,Honda-e (Recharging time: 36 Minutes; Power ch...,36,119,19.95,B,Honda-e
1,Nissan Leaf (Recharging time: 43 Minutes; Powe...,43,164,27.30,C,Nissan Leaf
2,Dacia Spring Electric (Recharging time: 38 Min...,38,108,17.50,A,Dacia Spring Electric
3,Fiat 500e (Recharging time: 25 Minutes; Power ...,25,164,26.11,B,Fiat 500e
4,Hyundai Kona Electric (Recharging time: 50 Min...,50,175,27.44,B,Hyundai Kona Electric
5,Tesla Model 3 (Recharging time: 25 Minutes; Po...,25,266,40.25,D,Tesla Model 3
6,Volkswagen ID.3 Pro S (Recharging time: 33 Min...,33,315,53.90,C,Volkswagen ID.3 Pro S
7,XPENG P5 (Recharging time: 44 Minutes; Power c...,44,269,42.00,D,XPENG P5


In [56]:

# pivot wider by power charged
model_data_wider = model_data.set_index('model_clean')['power_charged_(kw)'].T.to_frame().T

# clean column names
#model_data_wider.columns = model_data_wider.columns.str.extract(r'^([^\(]+)').squeeze().str.strip()

model_data_wider.columns = (
    model_data_wider.columns
    .str.replace(r'\s+', '_', regex=True)    # replace all whitespace with underscore
    .str.replace(r'_+$', '', regex=True)     # remove trailing underscores
    .str.replace(r'^_+', '', regex=True)     # remove leading underscores
    .str.lower()                             # convert to lowercase
    .str.strip()                            # just in case, strip spaces (optional)
)

#
# view
model_data_wider

model_clean,honda-e,nissan_leaf,dacia_spring_electric,fiat_500e,hyundai_kona_electric,tesla_model_3,volkswagen_id.3_pro_s,xpeng_p5
power_charged_(kw),19.95,27.3,17.5,26.11,27.44,40.25,53.9,42.0


In [57]:
print(model_data_wider.columns.tolist())


['honda-e', 'nissan_leaf', 'dacia_spring_electric', 'fiat_500e', 'hyundai_kona_electric', 'tesla_model_3', 'volkswagen_id.3_pro_s', 'xpeng_p5']


In [60]:
# recharge cost calculation - replicating EUAFO website

# create new dataframe to store all evs data by subsetting country dataframe
evs_data = country_data[['country_name', 'adhoc_energy_fee_dc']].copy()

# calc ev recharge estimation - replicating calculation on website
#model data[powercharged] x country_data[adhoc energyfee]
#evs_data['recharge_cost_nissan_leaf'] = evs_data['adhoc_energy_fee_dc'] * 27.3
evs_data['recharge_cost_nissan_leaf'] = evs_data['adhoc_energy_fee_dc'] * model_data_wider.at[model_data_wider.index[0], 'nissan_leaf']
evs_data['recharge_cost_tesla_model_3'] = evs_data['adhoc_energy_fee_dc'] * model_data_wider.at[model_data_wider.index[0], 'tesla_model_3']
evs_data['recharge_cost_volkswagen_id.3_pro_s'] = evs_data['adhoc_energy_fee_dc'] * model_data_wider.at[model_data_wider.index[0], 'volkswagen_id.3_pro_s']

evs_data = evs_data.round({'recharge_cost_nissan_leaf': 2, 'recharge_cost_tesla_model_3': 2,'recharge_cost_volkswagen_id.3_pro_s': 2})
evs_data

,country_name,adhoc_energy_fee_dc,recharge_cost_nissan_leaf,recharge_cost_tesla_model_3,recharge_cost_volkswagen_id.3_pro_s
0,Austria,0.516,14.09,20.77,27.81
1,Belgium,0.553,15.10,22.26,29.81
2,Bulgaria,0.485,13.24,19.52,26.14
3,Switzerland,0.525,14.33,21.13,28.30
4,Cyprus,0.650,17.74,26.16,35.04
5,Czech Republic,0.361,9.86,14.53,19.46
6,Germany,0.590,16.11,23.75,31.80
7,Denmark,0.582,15.89,23.43,31.37
8,Estonia,0.660,18.02,26.56,35.57
9,Spain,0.470,12.83,18.92,25.33


webscrape country info for ....
- new ev purchases (2025?)
- total cars on road
- total evs in fleet
- total dc charge points (these = fast ones)